# Tensorflow 불러오기

In [1]:
import tensorflow as tf

# MNIST 데이터셋 불러오기
x_train, x_test는 28×28 픽셀의 각 손글씨 이미지 데이터이고, y_train, y_test는 분류에 사용되는 0~9 사이의 레이블 값을 갖는다.

In [2]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 데이터 전처리
0~255.0 사이의 값을 갖는 픽셀값들을 0~1.0 사이의 값을 갖도록 변환한다.

In [3]:
X_train, X_test = X_train/255.0, X_test/255.0

# 모델 구성
tf.keras.models.Sequential()을 이용해서 인공신경망 모델을 구성한다.<br>
입력층(Input layer)에서 Flatten()을 이용해서 28×28 픽셀의 값을 784개의 1차원 배열로 변환한다.<br>
다음 두 개의 뉴런 층(Neuron layer)은 Dense()를 이용해서 완전 연결된 층(Fully-connected layer)을 구성한다.<br>
각 층은 512개와 10개의 인공 뉴런 노드를 갖고 활성화 함수(activation function)로는 각각 ReLU(tf.nn.relu)와 소프트맥스(tf.nn.softmax)를 사용한다.

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

Instructions for updating:
Colocations handled automatically by placer.


# 모델 컴파일
학습 과정에서 손실 함수(Loss function)를 줄이기 위해 사용되는 optimizer로는 Adam(Adaptive Momentum estimation)을 사용한다.<br>
손실 함수는 'sparse_categorical_crossentropy'를 지정하고, 평가 지표로는 정확도(accuracy)를 사용한다.<br>

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련
model.fit() 메서드에 학습 데이터와 레이블, 에포크를 순서대로 입력하면, 학습이 이루어진다.<br>
에포크(epoch)는 60,000개의 전체 학습 데이터를 몇 번 반복해서 학습할지를 의미한다.

In [6]:
%%time
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 9s 155us/sample - loss: 0.2041 - acc: 0.9391
Epoch 2/5
60000/60000 [==============================] - 9s 144us/sample - loss: 0.0811 - acc: 0.9749
Epoch 3/5
60000/60000 [==============================] - 8s 132us/sample - loss: 0.0536 - acc: 0.9835
Epoch 4/5
60000/60000 [==============================] - 9s 149us/sample - loss: 0.0375 - acc: 0.9881s - loss: 0.0375 - a
Epoch 5/5
60000/60000 [==============================] - 12s 192us/sample - loss: 0.0263 - acc: 0.9918
Wall time: 49.2 s


# 정확도 평가

In [7]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('테스트 정확도:', test_acc)

10000/10000 [==============================] - 1s 112us/sample - loss: 0.0712 - acc: 0.9778
테스트 정확도: 0.9778


In [8]:
%%time
loss, accuracy = [], []
for i in range(10):
    model.fit(X_train, y_train, epochs=1)
    loss.append(model.evaluate(X_test, y_test)[0])
    accuracy.append(model.evaluate(X_test, y_test)[1])

print(accuracy)

10000/10000 [==============================] - 1s 68us/sample - loss: 0.0961 - acc: 0.9823
[0.9793, 0.9803, 0.9757, 0.9789, 0.9803, 0.9806, 0.9819, 0.9813, 0.9798, 0.9823]
Wall time: 1min 40s


In [12]:
model.save('MNIST_DNN.h5')

# 손글씨 인식 프로그램

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import numpy as np
import tensorflow as tf

class MyApp(QMainWindow):

    def __init__(self):
        super().__init__()
        self.image = QImage(QSize(400, 400), QImage.Format_RGB32)
        self.image.fill(Qt.white)
        self.drawing = False
        self.brush_size = 30
        self.brush_color = Qt.black
        self.last_point = QPoint()
        self.loaded_model = None
        self.initUI()

    def initUI(self):
        menubar = self.menuBar()
        menubar.setNativeMenuBar(False)
        filemenu = menubar.addMenu('File')

        load_model_action = QAction('Load model', self)
        load_model_action.setShortcut('Ctrl+L')
        load_model_action.triggered.connect(self.load_model)

        save_action = QAction('Save', self)
        save_action.setShortcut('Ctrl+S')
        save_action.triggered.connect(self.save)

        clear_action = QAction('Clear', self)
        clear_action.setShortcut('Ctrl+C')
        clear_action.triggered.connect(self.clear)

        filemenu.addAction(load_model_action)
        filemenu.addAction(save_action)
        filemenu.addAction(clear_action)

        self.statusbar = self.statusBar()

        self.setWindowTitle('MNIST Classifier')
        self.setGeometry(300, 300, 400, 400)
        self.show()

    def paintEvent(self, e):
        canvas = QPainter(self)
        canvas.drawImage(self.rect(), self.image, self.image.rect())

    def mousePressEvent(self, e):
        if e.button() == Qt.LeftButton:
            self.drawing = True
            self.last_point = e.pos()

    def mouseMoveEvent(self, e):
        if (e.buttons() & Qt.LeftButton) & self.drawing:
            painter = QPainter(self.image)
            painter.setPen(QPen(self.brush_color, self.brush_size, Qt.SolidLine, Qt.RoundCap))
            painter.drawLine(self.last_point, e.pos())
            self.last_point = e.pos()
            self.update()

    def mouseReleaseEvent(self, e):
        if e.button() == Qt.LeftButton:
            self.drawing = False

            arr = np.zeros((28, 28))
            for i in range(28):
                for j in range(28):
                    arr[j, i] = 1 - self.image.scaled(28, 28).pixelColor(i, j).getRgb()[0] / 255.0
            arr = arr.reshape(-1, 28, 28)

            if self.loaded_model:
                pred = self.loaded_model.predict(arr)[0]
                pred_num = str(np.argmax(pred))
                self.statusbar.showMessage('숫자 ' + pred_num + '입니다.')

    def load_model(self):
        fname, _ = QFileDialog.getOpenFileName(self, 'Load Model', '')

        if fname:
            self.loaded_model = tf.keras.models.load_model(fname)
            self.statusbar.showMessage('Model loaded.')

    def save(self):
        fpath, _ = QFileDialog.getSaveFileName(self, 'Save Image', '', "PNG(*.png);;JPEG(*.jpg *.jpeg);;All Files(*.*) ")

        if fpath:
            self.image.scaled(28, 28).save(fpath)

    def clear(self):
        self.image.fill(Qt.white)
        self.update()
        self.statusbar.clearMessage()

In [2]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())

Instructions for updating:
Colocations handled automatically by placer.


SystemExit: 0

C:\Users\nakhe\Anaconda3\envs\tensor1_gpu\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
